HW3

# Project:
Implied Vol on each day:



In [ ]:
import numpy as np
import scipy.stats
from scipy.stats import norm
import scipy as sy
# Put Option on ED Futures:
# A call on the price is a put/floor on the implied rate, 
# so to analyze a ED Call, we have to use Black’s Put formula. 
# A put on the price is a call/cap on the implied rate, 
# so to analyze a ED Put, we have to use Black’s Call formula.
def N(x):
  value = scipy.stats.norm.cdf(x,0,1)
  # print('N(d) = ',value)
  return value
def NT(x):
  value = scipy.stats.norm.pdf(x,0,1)
  # print('NT(d) = ', value)
  return value
def DF(r, T):
  result = 1/(1+r*T)
  result2 = np.exp(-r*T)
  # print('DF = ', result)
  return result

def Black_evaluation(F, K, T, sigma, Black_Call = True): # act/365 year basis
  d = (F-K)/sigma/np.sqrt(T)
  return sigma*np.sqrt(T)*(NT(d)+d*N(d))

ED_alpha = 90/360
# Underlying rate, Exercise rate,TimetoMature, risk free rate, Volatility
def ED_Option(F, K, T, r, sigma, ED_Put = True):
  return DF(r,T)*1000000*Black_evaluation(F, K, T, sigma)*ED_alpha

def tick_value(ticks):
  return 25*ticks

def print_info(F, K, T, stub, sigma, tick):
  print('Given following on one day:')
  print('   F = ', F)
  print('   K = ', K)
  print('   time to expiry in years: ',T, 'years')
  print('   stub rate =', stub)
  print('   DF = ',DF(stub, T))
  print('When sigma = ', sigma,',')
  print(' calulated ED Option value = ', ED_Option(F, K, T, stub, sigma))
  print(' which is equal to the ED Put value of ',tick,'ticks = $', tick_value(tick))
  print('Therefore, Implied Normal Volatility sigma_N is then ',sigma, 'or ',round(sigma*10000,2),'bps')
  return ED_Option(F, K, T, stub, sigma)

ED_alpha = 90/360
K = (100-99.5)/100

# On Oct.8
print('----------Oct.8----------')
# sigma1 = implied_sig
F1 = (100-99.65)/100
T1 = 67/365 # time to expire
stub1 = 0.3/100
ticks1 = 5
# Put_value_Oct8 = print_info(F1, K, T1, stub1, sigma1, ticks1)

# On Oct.9
print('----------Oct.9----------')
# sigma2 = 0.61927/100
F2 = (100-99.6)/100
T2 = 66/365
stub2 = 0.35/100
ticks2 = 6.25
# Put_value_Oct9 = print_info(F2, K, T2, stub2, sigma2, ticks2)

----------Oct.8----------
----------Oct.9----------


Implied Vol:


In [ ]:
act_price_call = 5*25
act_price_call2 = 6.25*25

import scipy as sy
from scipy.stats import norm
def pricecall(F, K, sr, te, sig):
    d = (F-K)/sig/np.sqrt(te)
    dte = np.exp(-sr*te)
    nd = norm.cdf(d)
    ndp = norm.pdf(d)
    w = dte*1000000*sig*np.sqrt(te)*(ndp+d*nd)*1/4
    return w
sig_1 = sy.optimize.root(lambda x: pricecall(F1, K, stub1, T1, x)-act_price_call,0.0068).x[0]
print('Oct8 Implied Vol =',sig_1)
print('Oct8 Implied Break Even Daily Vol =',sig_1*np.sqrt(1/252))
sig_2 = sy.optimize.root(lambda x: pricecall(F2, K, stub2, T2, x)-act_price_call2,0.0068).x[0]
print('Oct9 Implied Vol =',sig_2)
print('Oct9 Implied Break Even Daily Vol =',sig_2*np.sqrt(1/252))

Oct8 Implied Vol = 0.0063772333006025085
Oct8 Implied Break Even Daily Vol = 0.00040172793728652037
Oct9 Implied Vol = 0.006192681317977952
Oct9 Implied Break Even Daily Vol = 0.00039010225514393725


Greeks on Oct8

In [ ]:
# greeks on Oct.8
# delta: change in price due to +1bp change in underlying contract price
print('----------Oct.8----------')
sigma1 = sig_1
contract1 = 99.65
contract2 = contract1 + 0.01

F1 = (100-contract1)/100
F2 = (100-contract2)/100

K = (100-99.5)/100
T1 = 67/365 # time to expire
stub1 = 0.3/100
ticks1 = 5

price = ED_Option(F1, K, T1, stub1, sigma1) #5cent
price_rate_bump = ED_Option(F1+0.0001, K, T1, stub1, sigma1)
delta = price_rate_bump-price
print('bump rate: delta = ',delta/25,'ticks, or $',delta)

 # price_contract_bump = ED_Option(F2, K, T1, stub1, sigma1)
# delta2 = price_contract_bump-price

# nagetive
delta_in_ticks = -delta/25
print(delta_in_ticks)

----------Oct.8----------
bump rate: delta =  0.2976615355796986 ticks, or $ 7.4415383894924645
-0.2976615355796986


In [ ]:
# Gamma: calculate two delta numbers (for two consecutive underlying bumps)
sigma1 = 0.638/100
F1 = (100-99.65)/100
T1 = 67/365 # time to expire
stub1 = 0.3/100
ticks1 = 5

price_1 = ED_Option(F1-0.0001, K, T1, stub1, sigma1)
price_2 = ED_Option(F1, K, T1, stub1, sigma1)
price_3 = ED_Option(F1+0.0001, K, T1, stub1, sigma1)

delta_1 = -(price_2-price_1)/25
delta_2 = -(price_3-price_2)/25

gamma = delta_2 - delta_1
print('bump rate: gamma = ',gamma)
#####################################################
F1 = (100-99.64)/100
F2 = (100-99.65)/100
F3 = (100-99.66)/100
price_1 = ED_Option(F1, K, T1, stub1, sigma1)
price_2 = ED_Option(F2, K, T1, stub1, sigma1)
price_3 = ED_Option(F3, K, T1, stub1, sigma1)

delta_1 = price_2-price_1
delta_2 = price_3-price_2

gamma2 = delta_2 - delta_1
print('bump contract: gamma = $',gamma2)

bump rate: gamma =  -0.012546862634898126
bump contract: gamma = $ 0.31367156588268585


In [ ]:
# vega: bump implied vol by 1bp
sigma1 = 0.638/100
F1 = (100-99.65)/100
T1 = 67/365 # time to expire
stub1 = 0.3/100
ticks1 = 5

price_1 = ED_Option(F1, K, T1, stub1, sigma1)
price_2 = ED_Option(F1, K, T1, stub1, sigma1+0.0001)
vega = price_2-price_1
print('vega = $',vega)

vega = $ 3.682316495839757


In [ ]:
# theta: change in time to expiry T by one day, i.e. -1/365 year
sigma1 = 0.638/100
F1 = (100-99.65)/100
T1 = 67/365 # time to expire
stub1 = 0.3/100
ticks1 = 5

price_1 = ED_Option(F1, K, T1, stub1, sigma1)
price_2 = ED_Option(F1, K, T1-1/365, stub1, sigma1)
theta = price_2-price_1
print('theta = $',theta)

theta = $ -1.7527207318067752


###PnLs contribution of Greeks


In [ ]:
ED_alpha = 90/360
K = (100-99.5)/100

# On Oct.8
print('----------Oct.8----------')
sigma1 = sig_1
contract1 = 99.65
F1 = (100-contract1)/100
T1 = 67/365 # time to expire
stub1 = 0.3/100
ticks1 = 5

# On Oct.9
print('----------Oct.9----------')
sigma2 = sig_2
contract2 = 99.6
F2 = (100-contract2)/100
T2 = 66/365
stub2 = 0.35/100
ticks2 = 6.25


# delta*25 is price change due to +1bp change in price (0.01 in contract1)
# (contract2-contract1)/(0.01) units -> (contract2-contract1)*100 number of 0.01 in the change
# delta*25*(F2-F1)*10000

deltaPL = delta_in_ticks*25*(contract2-contract1)*100
print('realized delta PnL = ',deltaPL)
deltaPL2 = delta/25*(F2-F1) # 0.29*5bp
print('realized delta PnL in ticks = ',deltaPL2)

# gamma is rate of change of delta, using taylor expansion to second order
# (F2-F1)/(1bp) units -> (F2-F1)*10000
dF = (F2-F1)*10000 # number of bps in the change 
gammaPL = 0.5*gamma*(dF)**2
print('realized gamma PnL = ',gammaPL)

# theta time decay is real put price change due to -1/365 year change in time to expiry T
# (T2-T1) years -> (T2-T1)/(-1/365) unit -> -(T2-T1)*365
# -theta*(T2-T1)*365
thetaPL = -theta*(T2-T1)*365 # should be the same as theta 
print('realized theta PnL = ',thetaPL)

# vega is real put price change due to +1bp change in vol(sigma)
# (sigma2-sigma1)/(1bp) -> (sigma2-sigma1)*10000
# vega*(sigma2-sigma1)*10000
vegaPL = vega*(sigma2-sigma1)*10000
print('realized vega PnL = ',vegaPL)


absolutePL = (156.25 - 125)
print('DeltaPnL is % 5.10f of total realized PnL.' %(deltaPL/absolutePL))
print('GammaPnL is % 5.10f of total realized PnL.' %(gammaPL/absolutePL))
print('thetaPnL is % 5.10f of total realized PnL.' %(thetaPL/absolutePL))
print('VegaPnL is % 5.10f of total realized PnL.' %(vegaPL/absolutePL))
print('Realized Total PnL =',absolutePL)

----------Oct.8----------
----------Oct.9----------
realized delta PnL =  37.20769194747079
realized delta PnL in ticks =  0.000148830767789883
realized gamma PnL =  3.920894573407452
realized theta PnL =  -1.7527207318067681
realized vega PnL =  -6.795788099583355
DeltaPnL is  1.1906461423 of total realized PnL.
GammaPnL is  0.1254686263 of total realized PnL.
thetaPnL is -0.0560870634 of total realized PnL.
VegaPnL is -0.2174652192 of total realized PnL.
Realized Total PnL = 31.25


In [ ]:
import pandas as pd
df = pd.DataFrame([['+1bp',str(round(delta_in_ticks,3))+' ticks','$'+str(round(deltaPL)),(deltaPL/absolutePL)], 
          ['-1bp,+1bp','$'+str(round(gamma,3)),'$'+str(round(gammaPL,3)),(gammaPL/absolutePL)], 
          ['-1/365','$'+str(round(theta,3)),'$'+str(round(thetaPL,3)),(thetaPL/absolutePL)], 
          ['+1bp','$'+str(round(vega,3)),'$'+str(round(vegaPL,3)),(vegaPL/absolutePL)]], 
          index=['delta', 'gamma', 'theta', 'vega'], columns=['bumping','Greeks value','P&L','pct on absolute PnL']).T
otherPL = absolutePL-(deltaPL+gammaPL+thetaPL+vegaPL)
df['otherPL'] = ['N\A','N\A','$'+str(round(otherPL,3)),otherPL/absolutePL]
df['sum'] = ['N\A','N\A','$'+str(round(deltaPL+gammaPL+thetaPL+vegaPL+otherPL,3)),
              df.loc['pct on absolute PnL'].sum()]
df

,delta,gamma,theta,vega,otherPL,sum
bumping,+1bp,"-1bp,+1bp",-1/365,+1bp,N\A,N\A
Greeks value,-0.298 ticks,$0.314,$-1.753,$3.682,N\A,N\A
P&L,$37.0,$3.921,$-1.753,$-6.796,$-1.33,$31.25
pct on absolute PnL,1.19065,0.125469,-0.0560871,-0.217465,-0.0425625,1


Amir Email:

In [ ]:
# On Oct.8
sigma1 = sig_1
contract1 = 99.65
F1 = (100-contract1)/100
K = (100-99.5)/100
T1 = 67/365
stub1 = 0.3/100

# On Oct.9
contract2 = 99.6
F2 = (100-contract2)/100
sigma2 = sig_2

price_1 = ED_Option(F1-0.0001, K, T1, stub1, sigma1)
price_2 = ED_Option(F1, K, T1, stub1, sigma1) #5cent
delta1 = (price_2-price_1)/25
print('minus rate: delta in ticks = ',delta1)

price_3 = ED_Option(F1+0.0001, K, T1, stub1, sigma1)
delta2 = (price_3-price_2)/25 # inticks
print('bump rate: delta in ticks = ',delta2)


gamma = delta2-delta1
print('gamma = ',gamma,'ticks')

minus rate: delta in ticks =  0.28511087022561865
bump rate: delta in ticks =  0.2976615355796986
gamma =  0.012550665354079948 ticks


In [ ]:
# for 1bp change in F rate, option increase 0.29 tick in price
# for actually 5bp change in F rate, option should increase
deltaPnL = delta2*(F2-F1)/(1/10000)
print('deltaPnL in ticks =', deltaPnL,' or ',deltaPnL*25,' in dollars')
print('overall PnL = 6.25-5 = 1.25 ticks, or ', 1.25*25,' in dollars')

deltaPnL in ticks = 1.4883076778988298  or  37.20769194747074  in dollars
overall PnL = 6.25-5 = 1.25 ticks, or  31.25  in dollars


In [ ]:
dF = (F2-F1) # 5bp
print(dF)

gammabp=gamma*10000
print('amir gamma =', gammabp)
gammabpPnL = 0.5*gammabp*10000*(dF)**2
print('gammabpPnL in ticks =',gammabpPnL)



gammaPnL = 0.5*gamma*(dF*10000)**2
print('gammaPnL in ticks =',gammaPnL,' or',gammaPnL*25,'in dollars')


0.0005000000000001132
amir gamma = 125.50665354079949
gammabpPnL in ticks = 0.1568833169260704
gammaPnL in ticks = 0.15688331692607044  or 3.922082923151761 in dollars


In [ ]:
price_4 = ED_Option(F1, K, T1, stub1, sigma1)
price_5 = ED_Option(F1, K, T1-1/365, stub1, sigma1)
theta = (price_5 - price_4)/25
print('theta in ticks = ',theta,'or ',theta*25,' in dollars')

thetaPnL = theta*(1/365)/(1/365)
print('thetaPnL in ticks =',thetaPnL,' or ', thetaPnL*25,'in dollars')

theta in ticks =  -0.07006921082428277 or  -1.7517302706070694  in dollars
thetaPnL in ticks = -0.07006921082428277  or  -1.7517302706070694 in dollars


In [ ]:
price_6 = ED_Option(F1, K, T1, stub1, sigma1)
price_7 = ED_Option(F1, K, T1, stub1, sigma1+0.0001)
vega = (price_7 - price_6)/25
print('vega in ticks =',vega, 'or ', vega*25,'in dollars')

vegaPnL = vega*(sigma2-sigma1)/(1/10000)
print('vegaPnL =',vegaPnL, 'or ',vegaPnL*25,'in dollars')

vega in ticks = 0.14727385765621365 or  3.6818464414053413 in dollars
vegaPnL = -0.271796824192209 or  -6.794920604805226 in dollars


In [ ]:
otherPnL = (6.25-5) - (deltaPnL + gammaPnL + thetaPnL + vegaPnL)
print('other PnL in ticks = ', otherPnL, 'or ',otherPnL*25, 'in dollar')

other PnL in ticks =  -0.053324959808408234 or  -1.3331239952102059 in dollar


In [ ]:
import pandas as pd
pd.options.display.float_format = "{:,.4f}".format
df = pd.DataFrame([['+1bp',delta2,deltaPnL,(deltaPnL/1.25)], 
          ['-1bp,+1bp',gamma,gammaPnL,(gammaPnL/1.25)], 
          ['-1/365',theta,thetaPnL,(thetaPnL/1.25)], 
          ['+1bp',vega,vegaPnL,(vegaPnL/1.25)]], 
          index=['delta', 'gamma', 'theta', 'vega'], columns=['bumping in Forward rate','Greeks(tick)','P&L(tick)','pct on realized PnL']).T
df['otherPnL'] = ['N\A','N\A',otherPnL,otherPnL/1.25]
df['Realized PnL'] = ['N\A','N\A',deltaPnL+gammaPnL+thetaPnL+vegaPnL+otherPnL,
              df.loc['pct on realized PnL'].sum()]
df

,delta,gamma,theta,vega,otherPnL,Realized PnL
bumping in Forward rate,+1bp,"-1bp,+1bp",-1/365,+1bp,N\A,N\A
Greeks(tick),0.2977,0.0126,-0.0701,0.1473,N\A,N\A
P&L(tick),1.4883,0.1569,-0.0701,-0.2718,-0.0533,1.2500
pct on realized PnL,1.1906,0.1255,-0.0561,-0.2174,-0.0427,1.0000


delta-hedge: short delta unit of underlying and long a put